In [1]:
import pandas as pd
import random

In [2]:
jumlah_cluster = 2
pangkat_pembobotan = 2
maksimum_iterasi = 10
toleransi_error = 0.01
fungsi_objektif = 0
iterasi_awal = 0

data_fuzzy = pd.read_csv('data_umur.csv')
data_fuzzy = data_fuzzy.drop(['Kelurahan', 'Total'], axis=1)
bil_random = pd.read_csv('bilangan_random.csv')

# data_fuzzy = pd.read_csv('data_kesehatan.csv')
# data_fuzzy = data_fuzzy.drop(['Kota / Kabupaten'], axis=1)
# bil_random = pd.read_csv('bilangan_random2.csv')

In [3]:
def find_uik(bil_random):
    uik2 = []
    for index_list, list_data in bil_random.iterrows():
        uikw2 = []
        for index_data, data in enumerate(list_data):
            hitung = round(data ** pangkat_pembobotan, 2)
            uikw2.append(hitung)
        uik2.append(uikw2)

    return uik2

In [27]:
def total_uikw(uik2):
    total_uiw = []
    # print(data_fuzzy.columns)

    for kolom in range(len(data_fuzzy.columns)):
        total = 0
        for data in uik2:
            total += data[kolom]
        #     print(data, kolom, total)
        # print()
            
        total_uiw.append(round(total,2))

    return total_uiw

In [5]:
def nama_uikw(bil_random):
    nama_uikw = []
    for i in range(len(bil_random.columns)):
        nama_uikw.append(f'ui{i+1}w2')
        
    return nama_uikw

In [6]:
def nama_uikxij(nama_uikw):
    nama_uikxij = []
    for i in range(len(nama_uikw)):
        for j in range(len(nama_uikw)):
            nama = f"ui{i+1}wxi{j+1}"
            nama_uikxij.append(nama)

    return nama_uikxij

In [7]:
def total_uikxij(uik2):
    uikxij = []

    for index_list, list_data in data_fuzzy.iterrows():
        uikw2xij : list = []
        for index_data, data in enumerate(uik2[index_list]):
            uikw2xij_list : list = []
            for index_data2, data2 in enumerate(list_data):
                hitung = round(data * data2, 2)
                uikw2xij_list.append(hitung)

            uikw2xij.extend(uikw2xij_list)
        uikxij.append(uikw2xij)

    return uikxij

In [8]:
def total_uiwxi(df_pusat_cluster, name_uikxij):
    total_uiwxi = []
    for nama in name_uikxij:
        total = round(df_pusat_cluster[nama].sum(), 2)
        total_uiwxi.append(total)

    return total_uiwxi

In [9]:
def find_clusterv(total_uiwxi, total_uiw):
    index_uiw = 0
    pusat_cluster = [[] for i in range(jumlah_cluster)]
    for index_uiwxi , data in enumerate(total_uiwxi):
        hitung = data / total_uiw[index_uiw]
        pusat_cluster[index_uiw].append(hitung)

        if (index_uiwxi + 1) % jumlah_cluster == 0:
            index_uiw += 1

    return pusat_cluster

In [10]:
def find_objektive(pusat_cluster):
    list_objektif = [[] for i in range(jumlah_cluster)]

    for index_data, data in enumerate(data_fuzzy.values):
        for index_cluster, cluster in enumerate(pusat_cluster):
            no_cluster = []
            for index_fuzzy, fuzzy in enumerate(data):
                hitung = (fuzzy - cluster[index_fuzzy]) ** 2

                no_cluster.append(hitung)
            list_objektif[index_cluster].append(no_cluster)

    return list_objektif

In [11]:
def create_df_center_cluster(uik2, uikxij, nama_uik2, nama_uikxij, bil_random):
    df_pusat_cluster = data_fuzzy.copy()
    df_pusat_cluster = df_pusat_cluster.merge(bil_random, left_index=True, right_index=True)
    df_pusat_cluster = df_pusat_cluster.merge(pd.DataFrame(uik2, columns=nama_uik2), left_index=True, right_index=True)
    df_pusat_cluster = df_pusat_cluster.merge(pd.DataFrame(uikxij, columns=nama_uikxij), left_index=True, right_index=True)

    return df_pusat_cluster

In [12]:
def create_df_cluster(list_objektif):
    df_cluster = []
    nama_kolom_cluster = [f"xi{i+1}_vi{i+1}" for i in range(jumlah_cluster)]
    nama_kolom_cluster.append('total')

    for index_objektif, objektif in enumerate(list_objektif):
        df_nilai = pd.DataFrame(columns=nama_kolom_cluster)
        for index_nilai, nilai in enumerate(objektif):
            total = sum(nilai)
            nilai.append(total)
            df_nilai.loc[index_nilai] = nilai
        df_cluster.append(df_nilai)

    return df_cluster

In [13]:
def count_function_objektive(df_cluster, uik2):
    p_cluster = []

    for index_cluster, cluster in enumerate(df_cluster):
        hitung_cluster = []
        for index_data, data in enumerate(cluster['total']):
            hitung = data * uik2[index_data][index_cluster]
            hitung_cluster.append(hitung)

        p_cluster.append(hitung_cluster)

    return p_cluster

In [14]:
def total_function_objektive(p_cluster):
    p_total = []

    for i in range(len(p_cluster[0])):
        total = 0
        for j in range(jumlah_cluster):
            total += p_cluster[j][i]

        p_total.append(total)

    return p_total

In [15]:
def sum_total_function_objektive(p_total):
    return sum(p_total)

In [16]:
def count_matriks(pusat_cluster):
    list_xijvkj2 = [[] for i in range(jumlah_cluster ** pangkat_pembobotan)]
    index_xij = 0
    for index_cluster, cluster in enumerate(pusat_cluster):
        for index_fuzzy, fuzzy in enumerate(data_fuzzy.columns):
            for index_data, data in enumerate(data_fuzzy[fuzzy]):
                hitung = (data - cluster[index_fuzzy]) ** 2
                # print(data, cluster[index_fuzzy], " = ", hitung)
                list_xijvkj2[index_xij].append(hitung)
            # print()
            index_xij += 1

    return list_xijvkj2

In [17]:
def total_matriks(list_xijvkj2):
    index_lt = 0
    total_lt = []
    for index_data, data in enumerate(list_xijvkj2[0]):
        total_angka = 0
        for index_list, list_total in enumerate(list_xijvkj2):
            total_angka += list_xijvkj2[index_list][index_data]
            
        total_lt.append(total_angka)

    return total_lt

In [18]:
def find_ui(total_lt, list_xijvkj2):
    ui = [[] for i in range(jumlah_cluster)]

    for i in range(jumlah_cluster):
        for index_data, data in enumerate(total_lt):
            angka1 = list_xijvkj2[2 * i][index_data]
            angka2 = list_xijvkj2[2 * i + 1][index_data]
            total_angka = angka1 + angka2
            hasil = round(total_angka / data, 2)
            ui[i].append(hasil)

    return ui

In [19]:
def check_optimal(hitung, toleransi_error):
    if hitung < toleransi_error:
        return True
    return False

In [31]:

def main():
    jumlah_cluster = 2
    pangkat_pembobotan = 2
    maksimum_iterasi = 30
    toleransi_error = 0.01
    fungsi_objektif = 0
    iterasi_awal = 0

    data_fuzzy = pd.read_csv('data_umur.csv')
    data_fuzzy = data_fuzzy.drop(['Kelurahan', 'Total'], axis=1)
    bil_random = pd.read_csv('bilangan_random.csv')

    # print(len(data_fuzzy.columns))

    for i in range(maksimum_iterasi):
        uik2 = find_uik(bil_random)
        uiw = total_uikw(uik2)
        name_uikw = nama_uikw(bil_random)
        name_uikxij = nama_uikxij(name_uikw)
        uikxij = total_uikxij(uik2)

        df_center_cluster = create_df_center_cluster(uik2, uikxij, name_uikw, name_uikxij, bil_random)
        uiwxi = total_uiwxi(df_center_cluster, name_uikxij)
        clusterV = find_clusterv(uiwxi, uiw)
        objektif = find_objektive(clusterV)
        df_cluster = create_df_cluster(objektif)
        hasil_objektif = count_function_objektive(df_cluster, uik2)
        p_total = total_function_objektive(hasil_objektif)
        sum_p = sum_total_function_objektive(p_total)
        list_xijvkj2 = count_matriks(clusterV)
        total_lt = total_matriks(list_xijvkj2)
        ui = find_ui(total_lt, list_xijvkj2)

        hitung = abs(sum_p - fungsi_objektif)
        print(f"|{sum_p}-{fungsi_objektif}|={hitung}")

        iterasi_awal = i- 1
        if not check_optimal(hitung, toleransi_error):
            bil_random = pd.DataFrame(ui).T
            
            fungsi_objektif = sum_p
            # print(bil_random)
        else :
            break
    print('selesai di iterasi', iterasi_awal)
    
main()

|212.31034019832188-0|=212.31034019832188
|191.9448033600611-212.31034019832188|=20.365536838260795
|190.59097739738527-191.9448033600611|=1.3538259626758133
|189.7158729438028-190.59097739738527|=0.8751044535824803
|189.18060681068107-189.7158729438028|=0.5352661331217234
|190.84759406733627-189.18060681068107|=1.6669872566552044
|190.1860802540791-190.84759406733627|=0.6615138132571872
|188.34677327102804-190.1860802540791|=1.8393069830510456
|189.42429562644168-188.34677327102804|=1.0775223554136346
|189.668927708196-189.42429562644168|=0.24463208175433238
|187.6766308780691-189.668927708196|=1.992296830126918
|188.00799009324547-187.6766308780691|=0.3313592151763771
|188.61616542793308-188.00799009324547|=0.6081753346876155
|188.65930701754382-188.61616542793308|=0.043141589610740994
|188.65930701754382-188.65930701754382|=0.0
selesai di iterasi 13
